
# ML-JET — Loss-Weight Sweep Aggregator (Notebook)

This notebook scans your sweep outputs under:

```
experiments/exp_loss_weight_sweep/training_output/
```

It collects per-run metrics from `training_summary.json`, builds a table with the columns needed for your paper, and then reports the **best run per model family** (ConvNeXt, EfficientNet, ViT, Swin, Mamba).

**Columns generated:**

- **Model (Initialization)** — inferred from `model_tag` / folder name
- **Batch** — `batch_size`
- **LR** — `learning_rate`
- **Total_Acc** — `best_accuracy`
- **Energy (Acc, Prec, Rec, F1)**
- **αs (Acc, Prec, Rec, F1)**
- **Q0 (Acc, Prec, Rec, F1)**

You can customize the ranking logic if needed.


In [4]:

# --- CONFIG ---
ROOT = "training_output/"   # path to run folders
OUT_DIR = ""                # where to write CSVs

# If running from a different CWD, you can set absolute paths:
# ROOT = "/wsu/home/.../experiments/exp_loss_weight_sweep/training_output"
# OUT_DIR = "/wsu/home/.../experiments/exp_loss_weight_sweep"


In [5]:

import os, json, re
from pathlib import Path
import pandas as pd

def load_json(p):
    try:
        with open(p, "r") as f:
            return json.load(f)
    except Exception as e:
        print(f"[WARN] Failed to read {p}: {e}")
        return None

FAMILY_PATTERNS = [
    (re.compile(r'efficientnet', re.I), 'EfficientNet'),
    (re.compile(r'convnext', re.I),     'ConvNeXt'),
    (re.compile(r'\bvit\b', re.I),      'ViT'),
    (re.compile(r'comer', re.I),        'ViT'),      # ViT-CoMer
    (re.compile(r'swin', re.I),         'Swin'),
    (re.compile(r'mamba', re.I),        'Mamba'),
]

def infer_family(model_tag: str, fallback: str) -> str:
    key = (model_tag or fallback or "").lower()
    for pat, fam in FAMILY_PATTERNS:
        if pat.search(key):
            return fam
    return "Other"

def get_nested(d, path, default=None):
    cur = d
    for k in path.split("."):
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

def tuple4(d, base):
    """Return (acc, prec, rec, f1) for a head or (None,..) if missing."""
    return (
        get_nested(d, f"{base}.accuracy"),
        get_nested(d, f"{base}.precision"),
        get_nested(d, f"{base}.recall"),
        get_nested(d, f"{base}.f1"),
    )

ROOT = Path(ROOT)
OUT_DIR = Path(OUT_DIR)
OUT_DIR.mkdir(parents=True, exist_ok=True)


In [6]:

rows = []
for run_dir in sorted(ROOT.iterdir()):
    if not run_dir.is_dir():
        continue
    summary_path = run_dir / "training_summary.json"
    if not summary_path.exists():
        continue

    js = load_json(summary_path)
    if js is None:
        continue

    model_tag = js.get("model_tag") or run_dir.name
    family = infer_family(model_tag, run_dir.name)

    # Scalars
    batch = js.get("batch_size")
    lr    = js.get("learning_rate")
    total_acc = js.get("best_accuracy")

    # Per-head tuples
    e_acc, e_prec, e_rec, e_f1 = tuple4(js, "best_model_metrics.energy")
    a_acc, a_prec, a_rec, a_f1 = tuple4(js, "best_model_metrics.alpha")
    q_acc, q_prec, q_rec, q_f1 = tuple4(js, "best_model_metrics.q0")

    rows.append({
        "family": family,
        "model_tag": model_tag,
        "run_dir": str(run_dir),
        "batch": batch,
        "lr": lr,
        "Total_Acc": total_acc,
        "Energy_Acc": e_acc, "Energy_Prec": e_prec, "Energy_Rec": e_rec, "Energy_F1": e_f1,
        "Alpha_Acc": a_acc,  "Alpha_Prec": a_prec,  "Alpha_Rec": a_rec,  "Alpha_F1": a_f1,
        "Q0_Acc": q_acc,     "Q0_Prec": q_prec,     "Q0_Rec": q_rec,     "Q0_F1": q_f1,
    })

import pandas as pd
df = pd.DataFrame(rows)
print(f"[OK] Collected {len(df)} runs from {ROOT}")
df.head()
display(df)

[OK] Collected 51 runs from training_output


,family,model_tag,run_dir,batch,lr,Total_Acc,Energy_Acc,Energy_Prec,Energy_Rec,Energy_F1,Alpha_Acc,Alpha_Prec,Alpha_Rec,Alpha_F1,Q0_Acc,Q0_Prec,Q0_Rec,Q0_F1
0,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S10_q0_alpha_pair,training_output/ConvNeXt_Gaussian_g500__lw-S10...,32,0.0001,0.680556,1.000000,1.000000,1.000000,1.000000,0.937500,0.939635,0.937500,0.937215,0.743056,0.742678,0.743056,0.742853
1,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S1_balanced,training_output/ConvNeXt_Gaussian_g500__lw-S1_...,32,0.0001,0.600000,1.000000,1.000000,1.000000,1.000000,0.950000,0.962963,0.952381,0.954751,0.650000,0.613636,0.625000,0.572917
2,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S1_balanced,training_output/ConvNeXt_Gaussian_g500__lw-S1_...,32,0.0001,0.670139,1.000000,1.000000,1.000000,1.000000,0.937500,0.939843,0.937500,0.937737,0.732639,0.735667,0.732639,0.702660
3,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S2_q0_mild,training_output/ConvNeXt_Gaussian_g500__lw-S2_...,32,0.0001,0.673611,1.000000,1.000000,1.000000,1.000000,0.937500,0.938154,0.937500,0.937228,0.736111,0.740431,0.736111,0.732660
4,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S3_q0_mid,training_output/ConvNeXt_Gaussian_g500__lw-S3_...,32,0.0001,0.677083,1.000000,1.000000,1.000000,1.000000,0.940972,0.941210,0.940972,0.941041,0.736111,0.753719,0.736111,0.736917
5,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S4_q0_strong,training_output/ConvNeXt_Gaussian_g500__lw-S4_...,32,0.0001,0.673611,1.000000,1.000000,1.000000,1.000000,0.934028,0.936282,0.934028,0.933963,0.739583,0.738285,0.739583,0.734323
6,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S5_q0_max,training_output/ConvNeXt_Gaussian_g500__lw-S5_...,32,0.0001,0.690972,1.000000,1.000000,1.000000,1.000000,0.937500,0.938389,0.937500,0.936980,0.753472,0.751971,0.753472,0.751455
7,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S6_alpha_mild,training_output/ConvNeXt_Gaussian_g500__lw-S6_...,32,0.0001,0.666667,1.000000,1.000000,1.000000,1.000000,0.934028,0.935758,0.934028,0.934113,0.732639,0.728995,0.732639,0.729871
8,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S7_alpha_strong,training_output/ConvNeXt_Gaussian_g500__lw-S7_...,32,0.0001,0.687500,1.000000,1.000000,1.000000,1.000000,0.930556,0.933683,0.930556,0.930459,0.756944,0.755805,0.756944,0.756322
9,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S8_energy_bump,training_output/ConvNeXt_Gaussian_g500__lw-S8_...,32,0.0001,0.687500,1.000000,1.000000,1.000000,1.000000,0.940972,0.943244,0.940972,0.940903,0.746528,0.748080,0.746528,0.747004


In [18]:

from os import path


full_csv = path.join(OUT_DIR, "aggregate_results_detailed.csv")
df.to_csv(full_csv, index=False)
print(f"[OK] wrote {full_csv}")

# Display (if running in a notebook UI that supports DataFrames)
try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("Loss-Weight Sweep — Full Results", df)
except Exception as e:
    print("[INFO] display_dataframe_to_user not available in this environment.")


[OK] wrote ./aggregate_results_detailed.csv
[INFO] display_dataframe_to_user not available in this environment.


In [19]:

# Rank by Total_Acc, tie-break on Q0_F1 (most challenging head)
df_numeric = df.copy()
for c in ["Total_Acc","Q0_F1"]:
    df_numeric[c] = pd.to_numeric(df_numeric[c], errors="coerce")

best_rows = []
for fam, sub in df_numeric.groupby("family"):
    sub = sub.copy()
    sub["_rank_key"] = list(zip(sub["Total_Acc"].fillna(-1), sub["Q0_F1"].fillna(-1)))
    best_idx = sub["_rank_key"].idxmax()
    best_rows.append(df.loc[best_idx])

best_df = pd.DataFrame(best_rows).sort_values("family")
best_csv = path.join(OUT_DIR, "best_per_family_detailed.csv")
best_df.to_csv(best_csv, index=False)
print(f"[OK] wrote {best_csv}")
best_df[["family","model_tag","batch","lr","Total_Acc",
         "Energy_Acc","Energy_Prec","Energy_Rec","Energy_F1",
         "Alpha_Acc","Alpha_Prec","Alpha_Rec","Alpha_F1",
         "Q0_Acc","Q0_Prec","Q0_Rec","Q0_F1"]]


[OK] wrote ./best_per_family_detailed.csv


,family,model_tag,batch,lr,Total_Acc,Energy_Acc,Energy_Prec,Energy_Rec,Energy_F1,Alpha_Acc,Alpha_Prec,Alpha_Rec,Alpha_F1,Q0_Acc,Q0_Prec,Q0_Rec,Q0_F1
6,ConvNeXt,ConvNeXt_Gaussian_g500__lw-S5_q0_max,32,0.0001,0.690972,1.0,1.0,1.0,1.0,0.937500,0.938389,0.937500,0.936980,0.753472,0.751971,0.753472,0.751455
18,EfficientNet,EfficientNet_g500__lw-S7_alpha_strong,32,0.0100,0.739583,1.0,1.0,1.0,1.0,0.954861,0.958062,0.954861,0.955222,0.784722,0.796988,0.784722,0.786834
24,Mamba,MambaOut_base_plus_rw_lrp_12_rlrp_4__lw-S3_q0_mid,16,0.0001,0.690972,1.0,1.0,1.0,1.0,0.944444,0.944934,0.944444,0.944621,0.746528,0.758722,0.746528,0.749777
43,Other,ViT_tiny_patch16_224_gaussian_lrp_60_rlrp_12_q...,32,0.0001,0.729167,1.0,1.0,1.0,1.0,0.961806,0.961941,0.961806,0.961821,0.767361,0.771524,0.767361,0.768891
37,Swin,Swin_g500__lw-S6_alpha_mild,32,0.0001,0.572917,1.0,1.0,1.0,1.0,0.895833,0.895856,0.895833,0.895731,0.677083,0.674624,0.677083,0.675676


In [20]:
# --- Add loss weights from run_dir name ---
import re

def parse_weights_from_dir(name: str):
    m = re.search(
        r'energy[_]?loss[_]?output[_-]?([0-9.]+).*?alpha[_]?output[_-]?([0-9.]+).*?q0[_]?output[_-]?([0-9.]+)',
        name,
        flags=re.I,
    )
    if m:
        return float(m.group(1)), float(m.group(2)), float(m.group(3))
    return None, None, None

best_df = best_df.copy()
best_df["w_energy"], best_df["w_alpha"], best_df["w_q0"] = zip(
    *best_df["run_dir"].map(parse_weights_from_dir)
)

# Save again with weights included
best_csv = path.join(OUT_DIR, "best_per_family_detailed_with_weights.csv")
best_df.to_csv(best_csv, index=False)
print(f"[OK] wrote {best_csv}")

# best_df[
#     ["family","model_tag","batch","lr","Total_Acc",
#      "Energy_Acc","Energy_Prec","Energy_Rec","Energy_F1",
#      "Alpha_Acc","Alpha_Prec","Alpha_Rec","Alpha_F1",
#      "Q0_Acc","Q0_Prec","Q0_Rec","Q0_F1",
#      "w_energy","w_alpha","w_q0"]
# ]
best_df[
    ["family",
     "Energy_Prec","Energy_Rec",
     "Alpha_Prec","Alpha_Rec",
     "Q0_Prec","Q0_Rec",
     "w_energy","w_alpha","w_q0"]
]


[OK] wrote ./best_per_family_detailed_with_weights.csv


,family,Energy_Prec,Energy_Rec,Alpha_Prec,Alpha_Rec,Q0_Prec,Q0_Rec,w_energy,w_alpha,w_q0
6,ConvNeXt,1.0,1.0,0.938389,0.937500,0.751971,0.753472,0.4,0.6,2.0
18,EfficientNet,1.0,1.0,0.958062,0.954861,0.796988,0.784722,0.6,1.6,0.8
24,Mamba,1.0,1.0,0.944934,0.944444,0.758722,0.746528,0.6,0.8,1.6
43,Other,1.0,1.0,0.961941,0.961806,0.771524,0.767361,0.8,0.8,1.4
37,Swin,1.0,1.0,0.895856,0.895833,0.674624,0.677083,0.8,1.2,1.0
